In this notebook I'll play a bit with the function multi_get_b_best, which computes the best block spectrum $b_{best}$ for a given vector $\boldsymbol{n} \in [0,1]^d$ and multipartition of $d$.

In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import sys
import os

# Add the src directory to the path to import our custom module
sys.path.insert(0, os.path.join('..', 'src'))

# Import our functions
from kaustav_conj.core import multi_get_b_best
from kaustav_conj.utils import list_all_partitions

In the following cell we import the function that maps $(\boldsymbol{n}, \boldsymbol{\lambda}) \mapsto \boldsymbol{b}_{best}$ for many partitions $\boldsymbol{\lambda}$ for which we know $\boldsymbol{b}_{best}$ analytically.

In [2]:
from kaustav_conj.ana_and_conj_bbest import b_best_ana

We now check that the optimizer `multi_get_b_best` does its job for $d=4$.

In [3]:
n_vectors = [[0.9, 0.3, 0.2, 0.1], [1., 0.7, 0.2, 0.1], [1., 0.9, 0.2, 0.1] ]
partitions_of_4 = list_all_partitions(4)
for partition in partitions_of_4:
    print(f"\n ==== Current partition: {partition} ====\n")
    for n in n_vectors:
        b_best_analytical = b_best_ana(n,partition)
        print(f"Analytical result: b_best = {b_best_analytical}")
        U_best, b_best_num, H_best = multi_get_b_best(n, partition, N_init=1, N_steps=400,learning_rate=0.01, verbosity=0)
        b_best_num = b_best_num.detach().numpy()
        print(f"Numerical result: b_best = {b_best_num}")
        print(f"Norm of difference: {np.linalg.norm(b_best_analytical - b_best_num)}\n")


 ==== Current partition: [3, 1] ====

Analytical result: b_best = [0.5 0.3 0.2 0.5]
Numerical result: b_best = [0.5 0.3 0.2 0.5]
Norm of difference: 3.6313094851731695e-09

Analytical result: b_best = [0.7  0.55 0.2  0.55]
Numerical result: b_best = [0.7  0.55 0.2  0.55]
Norm of difference: 1.730477717356947e-09

Analytical result: b_best = [0.9  0.55 0.2  0.55]
Numerical result: b_best = [0.9        0.54999998 0.2        0.55000002]
Norm of difference: 2.7021994301351995e-08


 ==== Current partition: [2, 2] ====

Analytical result: b_best = [0.5  0.25 0.5  0.25]
Numerical result: b_best = [0.5  0.25 0.5  0.25]
Norm of difference: 6.739249566527479e-09

Analytical result: b_best = [0.55 0.45 0.55 0.45]
Numerical result: b_best = [0.54999999 0.45000001 0.55000001 0.44999999]
Norm of difference: 1.4378588071374299e-08

Analytical result: b_best = [0.55 0.55 0.55 0.55]
Numerical result: b_best = [0.55002207 0.54998427 0.55001771 0.54997594]
Norm of difference: 4.0335170713097297e-05


 

Now for $d=5$. TO BE COMPLETED...

In [21]:
# generate 10 random 5-component vectors and sort each in decreasing order
n_vectors = [np.sort(np.random.rand(5))[::-1].tolist() for _ in range(10)]
partitions_of_5 = [[3,1,1], [3,2]] #TODO
for partition in partitions_of_5:
    print(f"\n ==== Current partition: {partition} ====\n")
    for n in n_vectors:
        b_best_analytical = b_best_ana(n,partition)
        print(f"Analytical result: b_best = {b_best_analytical}")
        U_best, b_best_num, H_best = multi_get_b_best(n, partition, N_init=1, N_steps=1000,learning_rate=0.01, verbosity=0)
        b_best_num = b_best_num.detach().numpy()
        print(f"Numerical result: b_best = {b_best_num}")
        print(f"Norm of difference: {np.linalg.norm(b_best_analytical - b_best_num)}\n")


 ==== Current partition: [3, 1, 1] ====

Analytical result: b_best = [0.43505461 0.43505461 0.40900759 0.43505461 0.43505461]
Numerical result: b_best = [0.43505462 0.43505461 0.40900759 0.43505461 0.43505461]
Norm of difference: 4.61751636220232e-09

Analytical result: b_best = [0.5371972  0.41021292 0.41021292 0.41021292 0.41021292]
Numerical result: b_best = [0.5371972  0.41021292 0.41021291 0.41021292 0.41021292]
Norm of difference: 6.166954774166014e-09

Analytical result: b_best = [0.37539278 0.31620997 0.31620997 0.31620997 0.31620997]
Numerical result: b_best = [0.37539278 0.31620998 0.31620995 0.31620998 0.31620997]
Norm of difference: 1.9765903925262517e-08

Analytical result: b_best = [0.2658733  0.2658733  0.21890157 0.2658733  0.2658733 ]
Numerical result: b_best = [0.26587795 0.26585959 0.21890143 0.2658783  0.26587748]
Norm of difference: 1.5880519908081343e-05

Analytical result: b_best = [0.6062373  0.6062373  0.54370613 0.6062373  0.6062373 ]
Numerical result: b_best

ValueError: Sorry, for the partition you chose the function has not been implemented yet!